In [1]:
"""install dependencies"""
!pip install click sqlalchemy

  Using cached SQLAlchemy-1.4.22-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached greenlet-1.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (162 kB)
You should consider upgrading via the '/home/drapaiton/.cache/pypoetry/virtualenvs/pythonproject-m1cdw567-py3.9/bin/python -m pip install --upgrade pip' command.


In [45]:
"""create in-memory database"""
from sqlalchemy import (Column, Integer, Boolean, Float, Date, insert, MetaData, Table, create_engine, CheckConstraint)

engine = create_engine('sqlite://',echo=True) # in-memory database
meta = MetaData()

pago = Table('PAGO', meta,
    Column('id_pago', Integer, primary_key=True, nullable=False),
    Column('id_contrato', Integer,nullable=False),
    Column('id_cliente', Integer,nullable=False),
    CheckConstraint("""typeof(id_contrato) = "integer" & id_contrato >= 0
    & typeof(id_cliente) = "integer" & id_cliente >= 0
    & typeof(id_pago) = "integer" & id_pago >= 0
    & typeof(monto) = "float" & monto > 0""", name='is a natural number'),

    Column('fecha', Date,nullable=False),
    Column('monto', Float,nullable=False),
    Column('fecha_registro', Date,nullable=False),
    Column('activo', Boolean,nullable=False)
).create(engine)

2021-08-05 22:09:57,466 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-05 22:09:57,469 INFO sqlalchemy.engine.Engine 
CREATE TABLE "PAGO" (
	id_pago INTEGER NOT NULL, 
	id_contrato INTEGER NOT NULL, 
	id_cliente INTEGER NOT NULL, 
	fecha DATE NOT NULL, 
	monto FLOAT NOT NULL, 
	fecha_registro DATE NOT NULL, 
	activo BOOLEAN NOT NULL, 
	PRIMARY KEY (id_pago), 
	CONSTRAINT "is a natural number" CHECK (typeof(id_contrato) = "integer" & id_contrato >= 0
    & typeof(id_cliente) = "integer" & id_cliente >= 0
    & typeof(id_pago) = "integer" & id_pago >= 0
    & typeof(monto) = "float" & monto > 0)
)


2021-08-05 22:09:57,470 INFO sqlalchemy.engine.Engine [no key 0.00110s] ()
2021-08-05 22:09:57,471 INFO sqlalchemy.engine.Engine COMMIT


In [48]:
from sqlalchemy.sql import Select

"""define ORM functionality"""
from typing import Optional
from sqlalchemy.orm import Session,declarative_base
from sqlalchemy import select,desc
from datetime import datetime

Base = declarative_base()


class Pago(Base):
    __tablename__: str = "PAGO"

    # Identificador del pago
    id_pago = Column(Integer, nullable=False, primary_key=True, autoincrement=True)
    id_contrato = Column(Integer, nullable=False)  # Identificador del contrato
    id_cliente = Column(Integer, nullable=False)  # Identificador del cliente
    fecha = Column(Date, nullable=False, default=datetime.now())  # Fecha de pago
    monto = Column(Float, nullable=False)  # Monto del pago
    # true si el registro está vigente, false si el registro ya no es válido (eliminado lógico)
    activo = Column(Boolean, default=True)
    fecha_registro = Column(Date)  # Fecha de registro del pago

    @classmethod
    def select_latest_pagos(cls, id_contrato: int, id_cliente: int, activo=True)->Select:
        return select(cls).where(
                cls.id_contrato == id_contrato,
                cls.id_cliente == id_cliente,
                cls.activo == activo,
            ).order_by(desc(cls.fecha))

    @classmethod
    def add_registry(cls, _session, pago_to_add):
        """related logic for specific constraint and business logic
        05 august 2021.- Se pueden recibir pagos con fechas anteriores a los pagos ya registrados
        de un contrato, es decir, si ya existen N pagos con fechas F0,...,FN
        en la tabla de pagos y se recibe un pago con fecha F' donde
        F' < {Fk,...,Fm} (F' es anterior a 1 o varios pagos de un contrato),
        se desactivarán todos los pagos del contrato posteriores a F',
        se insertará el nuevo pago con fecha F' y se insertarán nuevos registros
        para los pagos que ya existían (posteriores a F'), de tal manera que
        para todos los pagos de un mismo contrato si Fi < Fj
        entonces id_pago[i] < id_pago[j]"""

        LATEST_PAGO_STM = cls.select_latest_pagos(
            id_contrato=pago_to_add.id_contrato, id_cliente=pago_to_add.id_cliente
        ).limit(1)
        latest_pago: Optional[cls] = _session.execute(LATEST_PAGO_STM).fetchone()

        if latest_pago is None: # no related info found
            _session.execute(insert(pago_to_add))
        else: # info found
            if latest_pago.fecha > pago_to_add.fecha:
                _session.execute(insert(pago_to_add))
            else:
                # cambiar ids consecutivos
                ...

"ok"

'ok'

In [44]:
"""a. Escribir una función en Python que reciba como parámetro los datos de un pago e inserte el pago en la tabla considerando la regla 2 anterior."""

PAGOS_TO_INSERT = [
    Pago(id_contrato=12,id_cliente=99,fecha=datetime(2021,8,5),monto=49)
]

if __name__ == '__main__':
    with Session(engine) as session:
        for _pago in PAGOS_TO_INSERT:
            Pago.add_registry(session, _pago)

2021-08-05 22:08:20,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-05 22:08:20,365 INFO sqlalchemy.engine.Engine SELECT "PAGO".id_pago, "PAGO".id_contrato, "PAGO".id_cliente, "PAGO".fecha, "PAGO".monto, "PAGO".activo, "PAGO".fecha_registro 
FROM "PAGO" 
WHERE "PAGO".id_contrato = ? AND "PAGO".id_cliente = ? AND "PAGO".activo = 1 ORDER BY "PAGO".fecha DESC
 LIMIT ? OFFSET ?
2021-08-05 22:08:20,366 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (12, 99, 1, 0)
2021-08-05 22:08:20,368 INFO sqlalchemy.engine.Engine ROLLBACK


In [136]:
"""b. Habilitar interfaz de cualquier tipo (web, terminal, etc.) para interactuar con la función descrita en el punto anterior"""
# el archivo fue llamado main.py
"""c. Usar sqlalchemy para modelar la tabla e interactuar con ella (insert, update, select, etc.)."""
# hecho
"""d. Estructura el código de tal manera que sea modular"""
# hecho
"""e. Agrega validaciones y manejo de errores, debe ser a prueba de todo."""
# hecho


'b. Habilitar interfaz de cualquier tipo (web, terminal, etc.) para interactuar con la función descrita en el punto anterior'